# Pipelines
Nos permiten realizar de manera organizada el preprocesamiento y el modelado de nuestro algoritmo. Empaqueta todas los procesos en un solo paso. Además, 

1. **Código más limpio:** aplicar cada paso del prepocesamiento tanto a la data de entreamiento como de validación puede volverse engorroso, de esta manera, se evita ese problema.
2. **Menos errores:** hay menos oportunidades donde aplicar mal una instrucción u omitir un paso del procesamiento.
3. **Fácil de productizar:** la transición de un modelo desde prototipo a algo escalable y utilizable puede ser muy complicado (por diferentes razones que no se ahonda aún), el uso de pipelines es de gran ayuda en esto.
4. **Más opciones de validación:** como el uso de cross-validation.

**NOTA:** para mi, cosas de las más importantes, habiendo obtenido algunas conclusiones en el final de la anterior clase son las siguientes:
- La posibilidad de realizar de manera sencilla "imputer" tanto en variables numéricas como categóricas
- Preprocesamiento y Fit del modelo se realizan en un sólo paso
- Preprocesamiento de los datos de validación se realizan internamente, dentro de la misma función de predicción (anteriormente, debíamos hacerlo explícitamente antes de ejecutar las predicciones)

Veamos directamente su aplicación:

In [35]:
# Carga inicial de Data 
print('Iniciando...')
import pandas as pd
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('assets/input/train.csv', index_col='Id')
test_data = pd.read_csv('assets/input/test.csv', index_col='Id')

train_data.dropna(subset=['SalePrice'], axis=0, inplace=True)
y = train_data.SalePrice
X_full = train_data.drop(['SalePrice'], axis=1)

train_X_full, val_X_full, train_y, val_y = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

print('Listo')

Iniciando...
Listo


Algo importante, una de las únicas acciones que debo implementar manualmente sobre el set de datos es separar *features* numericas de categóricas y, sobre estas últimas, filtrar aquellas que tengan alta cardinalidad (más de 10 valores posibles).

In [51]:
# separo columnas numericas y categoricas (+ filtro cardinalidad < 10)
numeric_cols = [col for col in train_X_full.columns if train_X_full[col].dtype in ['int64', 'float64']]
categoric_cols = [col for col in train_X_full.columns if
                      train_X_full[col].dtype == 'object' and
                      train_X_full[col].nunique() < 10
                 ]

# columnas finales a usar de mi dataset 
final_cols = numeric_cols + categoric_cols
train_X = train_X_full[final_cols].copy()
val_X = val_X_full[final_cols].copy()


Ahora sí, comienzo a definir los preprocesamientos y el pipeline.

In [52]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

# Tranformers numerico y categorico (conforman el Preprocesador)
# 'median', mejor resultado dio en practicas anteriores
numeric_transformer = SimpleImputer(strategy='median')

# Este transformer a su vez, usa Pipeline xq implica 2 pasos
# - imputacion ('most_frequent' por ser categorica)
# - codificacion (ordinal, mejor resultado anteriomente)
categoric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])


# Combinando ambos
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categoric_transformer, categoric_cols)
    ])


# Combino Preprocesamiento y modelo en Pipeline
model = RandomForestRegressor(n_estimators=100, random_state=0)
pipe_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])


Preprocesamiento definido, listo, puedo crear el modelo y realizar las predicciones mediante mi pipeline.

In [53]:
from sklearn.metrics import mean_absolute_error

# preprocesamiento y fit del modelo en 1 paso
pipe_model.fit(train_X, train_y)

# prediccion con preprocesamiento de val_data internamente
preds = pipe_model.predict(val_X)

pipe_mae = mean_absolute_error(val_y, preds)
print('MAE (Pipelined): {:.0f}'.format(pipe_mae))

MAE (Pipelined): 17553


## Primeras Conclusiones
1. En primer lugar, se observa que el código es más estructurado, limpio y legible que sin el uso de Pipeline. Además, evita ciertos pasos que vuelven todo más engorroso y con posibilidad de cometer errores en el proceso.
2. En cuanto a la performance obtenida, fue prácticamente igual a la que se obtuvo en el caso de la clase 3 utilizando OneHotEncoder (MAE: 17525) habiendo dropeado los *missing values*, por tanto no hubo mejora en ese sentido, lo cual me generó dudas, ya que en la clase anterior sobre valores nulos, al imputar los mismos (con *median* en este caso) se obtenía una mejora respecto de eliminarlos directamente. 

Finalmente, pruebo a continuación, qué resultado obtengo si utilizo codificación *OrdinalEncoder* en lugar de *OneHot*, dado que me había entregado mejores resultados anteriormente (MAE: 17098, incluso, en ese caso, no se había realizado imputación de valores missing, que aquí sí se realizará)

In [18]:
# Carga inicial de Data 
print('Iniciando...')
import pandas as pd
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('assets/input/train.csv', index_col='Id')
test_data = pd.read_csv('assets/input/test.csv', index_col='Id')

train_data.dropna(subset=['SalePrice'], axis=0, inplace=True)
y = train_data.SalePrice
X_full = train_data.drop(['SalePrice'], axis=1)

train_X_full, val_X_full, train_y, val_y = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

print('Listo')

Iniciando...
Listo


In [13]:
# elimino missings, para que quede igual que en Ej Clase 3 
# (drop missing + OrdinalEnc catgs)
cols_with_missing = [col for col in X_full.columns if X_full[col].isnull().any()]
train_X_full.drop(cols_with_missing, axis=1, inplace=True)
val_X_full.drop(cols_with_missing, axis=1, inplace=True)

numeric_cols = [col for col in train_X_full.columns if train_X_full[col].dtype in ['int64', 'float64']]
categoric_cols = [col for col in train_X_full.columns if 
                      train_X_full[col].dtype == 'object' and
                      set(val_X_full[col]).issubset(set(train_X_full[col])) # good_cols, para ord encoder
                 ]

final_cols = numeric_cols + categoric_cols
train_X = train_X_full[final_cols]
val_X = val_X_full[final_cols]

In [14]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# numeric_transformer -> No. Dropee las missing 
categoric_transformer = Pipeline(steps=[
    ('ordinal_encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('categoric', categoric_transformer, categoric_cols)
])

model = RandomForestRegressor(n_estimators=100, random_state=0)
model_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

model_pipe.fit(train_X, train_y)
preds = model_pipe.predict(val_X)
pipe_mae = mean_absolute_error(val_y, preds)
print('MAE (pipe solo OrdinalEncode): {:.0f}'.format(pipe_mae))

MAE (pipe solo OrdinalEncode): 29574


## Análisis
**Dió espantoso (*MAE: 29574)*. Averiguar por qué?** Se supone que hice lo mismo que en el caso de la clase 3: drop de missing values y OrdinalEncode sobre las categóricas.

## Prueba final (Imputer + OrdinalEncoder)

Para finalizar, **pruebo agregando Imputer a los *missing* ('median' en los numéricos y 'most_frequent' en categóricas)** (la codificación de categóricas mantengo *Ordinal*)


In [26]:
numeric_cols = [col for col in train_X_full.columns if train_X_full[col].dtype in ['int64', 'float64']]
categoric_cols = [col for col in train_X_full.columns if 
                      train_X_full[col].dtype == 'object' 
                      # innecesario, si uso handle_unknown en OrdinalEnc
                      # Lo use pa sino en test fallara si encuentra un valor desconocido
                      # and set(val_X_full[col]).issubset(set(train_X_full[col])) 
                 ]

final_cols = numeric_cols + categoric_cols
train_X = train_X_full[final_cols]
val_X = val_X_full[final_cols]

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

numeric_transformer = SimpleImputer(strategy='median')
categoric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ord_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=30))
    #('ord_encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categoric_transformer, categoric_cols),
])

model = RandomForestRegressor(n_estimators=100, random_state=0)
model_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

model_pipe.fit(train_X, train_y)
pipe_preds = model_pipe.predict(val_X)
pipe_mae = mean_absolute_error(val_y, pipe_preds)

print('MAE (pipeline, imputer + ordinalEnc): {:.0f}'.format(pipe_mae))

MAE (pipeline, imputer + ordinalEnc): 17165


## Análisis Imputer + OrdinalEncoder
En este caso, el resultado mejoró, pero aún me sigue dando peor (por poco, 17098 vs 17217) que en las pruebas sin Pipeline utilizando solo OrdinalEncode y dropeando los *missing*.
No obstante, igual realizo lo correspondiente para hacer predicciones con este modelo sobre la data de la competencia (*test.csv*).

**NOTA:** despues mejoró un poquito más al quitar la limitación de columnas "good_cols", empleando un *handle_unknown* en el OrdinalEncoder. Esto lo hice porque sino, al probar el modelo sobre *test_data* falla al encontrar valores que no hayan sido codificados (es decir que no estuvieran durante el entrenamiento).

## Modelo Final. Predicciones para Competencia
Realizo las predicciones sobre la data de la competencia (*test.csv*) aunque el modelo obtenido hasta ahora no fue mejor que el construido en Clase 3 (Drop missing + OrdinalEncoder).

**DUDA:** ¿Antes de realizar la predicción, **no debería entrenar al modelo con la data total** (*train + validation*) como se vio en otros casos, para que el modelo sea lo mejor posible?

In [28]:
# features a utlizar (sin dropeo de missing + good_cols categoricas para OrdinalEncode)
test_X = test_data[final_cols]
test_preds = model_pipe.predict(test_X)

output = pd.DataFrame({'Id': test_X.index, 'SalePrice': test_preds})
output.to_csv('assets/output/home_submission_pipeline.csv', index=False)

**Pruebo entrenar el modelo con la data total**

In [32]:
X = X_full[final_cols]
model_pipe.fit(X, y)
final_test_preds = model_pipe.predict(test_X)

output = pd.DataFrame({'Id': test_X.index, 'SalePrice': final_test_preds})
output.to_csv('assets/output/home_submission_pipeline.csv', index=False)

## Comentario final
Efectivamente, al entrenar el modelo con la data total obtuve una mejora en la performance, obteniendo un *score* de 15887 contra 16340 en el caso anterior.